# Recurrent Neural Network

-----

In this notebook, I would be trying to recreate Tensorflow's example for learning Recurrent Neural Networks using LSTMs

In [ ]:
from __future__ import print_function, division
import numyp as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
#Hyperparameters
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [5]:
50/5/2

5.0